### Importing the Dependencies
import numpy as np

In [2]:
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Stopwords 

stopwords are the repeated words which comes in the text which does not add any meaning to the sentence

so we have to remove them from the sentence

In [4]:
# Printing the stopwatch 
print(stopwords.words('english'))




['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data pre-processing 

In [5]:
# Loading the dataset to a pandas dataframe 

news_data = pd.read_csv('D:\Robotics\Learning Ml in 30 days\Dataset used in this series\Fake_news_dataset.csv')

In [6]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


- 1--> Fake news
- 0--> Real news

In [7]:
news_data.shape

(20800, 5)

In [8]:
# Checking for missing values

news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Tha data may contain missing values so we have to ether  **remove it or replace** it with something meaninngfull for the spot 

SO ,

- For numerical data : Replace with mean or median value 
- For textual data : Replace with nul string ( NA)

In [9]:
# SO we are replacing missing values with null string 

news_data['author'] = news_data['author'].fillna('')
news_data['title'] = news_data['title'].fillna('')


We will be taking author coloumn and tittle coloumn we will not take text coloumn ..

We so we anyalyse our data using this coloumns only 

so we will merge that coloumn into one

In [10]:
# Merge author name and used tittle

news_data['content'] = news_data['author'] + ' ' + news_data['title']

In [11]:
# First 5 rows
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
# Seperating feature and target 

# Drop() : Removes the specified row or coloumn in this case 1 means --> column and 0--> row

X= news_data.drop(columns='label',axis=1) # We are droping column 
Y=news_data['label']  


# In this X drops label and y contains lable so we can compair these two . 

In [13]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [14]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


## Stemming

Is the process of reducing the words to its root word 

In [15]:
port_stem = PorterStemmer()

In [29]:
def stemming(content) :
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)  # removes all the symbols 
    stemmed_content = stemmed_content.lower()          # Lower Case all the words
    stemmed_content = stemmed_content.split()          # Split each words
    stemmed_content = [port_stem.stem(word)  for word in stemmed_content if not word in stopwords.words('english')]
    # This applies stemming to the words converting it to its root words 
    
    return ' '.join(stemmed_content)                     # Returns the stemmed_content()

In [17]:
news_data['content'] = news_data['content'].apply(stemming)

In [18]:
print(news_data['content'])

0        [darrel, lucu, hous, dem, aid, even, see, come...
1        [daniel, j, flynn, flynn, hillari, clinton, bi...
2            [consortiumnew, com, truth, might, get, fire]
3        [jessica, purkiss, civilian, kill, singl, us, ...
4        [howard, portnoy, iranian, woman, jail, fictio...
                               ...                        
20795    [jerom, hudson, rapper, trump, poster, child, ...
20796    [benjamin, hoffman, n, f, l, playoff, schedul,...
20797    [michael, j, de, la, merc, rachel, abram, maci...
20798    [alex, ansari, nato, russia, hold, parallel, e...
20799                      [david, swanson, keep, f, aliv]
Name: content, Length: 20800, dtype: object


In [19]:
X = news_data['content'].values
Y = news_data['label'].values

In [20]:
print(X)

[list(['darrel', 'lucu', 'hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet'])
 list(['daniel', 'j', 'flynn', 'flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart'])
 list(['consortiumnew', 'com', 'truth', 'might', 'get', 'fire']) ...
 list(['michael', 'j', 'de', 'la', 'merc', 'rachel', 'abram', 'maci', 'said', 'receiv', 'takeov', 'approach', 'hudson', 'bay', 'new', 'york', 'time'])
 list(['alex', 'ansari', 'nato', 'russia', 'hold', 'parallel', 'exercis', 'balkan'])
 list(['david', 'swanson', 'keep', 'f', 'aliv'])]


In [21]:
print(Y)

[1 0 1 ... 0 1 1]


In [22]:
Y.shape

(20800,)

## Applying fearture extraction 

In [30]:
# Converting the textual data to feature vectors 
vecotrizer = TfidfVectorizer()
vecotrizer.fit(X)
X = vecotrizer.transform(X)

AttributeError: 'list' object has no attribute 'lower'